# Quantitative Momentum Investing Strategy

In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
from scipy import stats #The SciPy stats module
import math
yf.pdr_override()

In [2]:
sp500 = pd.read_csv('s&p500_tickers.csv')
del sp500['Unnamed: 0']

In [3]:
print(sp500['Tickers'])

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Tickers, Length: 503, dtype: object


In [4]:
#Datetime object for current date
from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
today 

'2022-08-09'

In [5]:
#Datetime object for one month ago
one_month = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')

In [6]:
#Datetime object for three month ago
three_months = (datetime.today() - timedelta(days=91)).strftime('%Y-%m-%d')

In [7]:
#Datetime object for six month ago
six_months = (datetime.today() - timedelta(days=183)).strftime('%Y-%m-%d')

In [8]:
#Datetime object for one year ago
one_year = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')

## Building a base case before iteration for return metrics

In [9]:
tickers = sp500['Tickers'][0]
start_date = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')
price = round(yf.download(tickers, start=start_date, end=today)['Adj Close'][0],2)
price

[*********************100%***********************]  1 of 1 completed


148.48

In [10]:
tickers = sp500['Tickers'][1]
start_date = one_month
end_date = today
port = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
port_return = (port / port.shift(1)) - 1
one_month_return = round(port_return.mean() * 30, 6)
one_month_return

[*********************100%***********************]  1 of 1 completed


0.135791

In [11]:
tickers = sp500['Tickers'][1]
start_date = three_months
end_date = today
port = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

port_return = (port / port.shift(1)) - 1
three_month_return = round(port_return.mean() * 91, 6)
three_month_return

[*********************100%***********************]  1 of 1 completed


0.068158

In [12]:
tickers = sp500['Tickers'][1]
start_date = six_months
end_date = today
port = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

port_return = (port / port.shift(1)) - 1
six_month_return = round(port_return.mean() * 183, 6)
six_month_return

[*********************100%***********************]  1 of 1 completed


-0.216875

In [13]:
tickers = sp500['Tickers'][1]
start_date = one_year
end_date = today
port = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

port_return = (port / port.shift(1)) - 1
annual_returns = round(port_return.mean() * 252,6)
annual_returns

[*********************100%***********************]  1 of 1 completed


-0.087659

# Iterating through S&P 500 tickers

In [14]:
#Empty dataframe to store data
my_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]
hqm_dataframe = pd.DataFrame(columns = my_columns)

In [15]:
hqm_dataframe

Empty DataFrame
Columns: [Ticker, Price, Number of Shares to Buy, One-Year Price Return, One-Year Return Percentile, Six-Month Price Return, Six-Month Return Percentile, Three-Month Price Return, Three-Month Return Percentile, One-Month Price Return, One-Month Return Percentile, HQM Score]
Index: []

In [16]:
for t in sp500['Tickers']:
    try:
        tickers = t
        start_date = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')
        price = round(yf.download(tickers, start=start_date, end=today)['Adj Close'][0],2)

        daily_one_year = yf.download(tickers, start=one_year, end=today)['Adj Close']
        daily_one_year_return = (daily_one_year / daily_one_year.shift(1)) - 1
        one_year_return = round(daily_one_year_return.mean() * 252,6)

        daily_six_month = yf.download(tickers, start=six_months, end=today)['Adj Close']
        daily_six_month_return = (daily_six_month / daily_six_month.shift(1)) - 1
        six_month_return = round(daily_six_month_return.mean() * 183, 6)

        daily_three_month = yf.download(tickers, start=three_months, end=today)['Adj Close']
        daily_three_month_return = (daily_three_month / daily_three_month.shift(1)) - 1
        three_month_return = round(daily_three_month_return.mean() * 91, 6)

        daily_one_month = yf.download(tickers, start=one_month, end=today)['Adj Close']
        daily_one_month_return = (daily_one_month / daily_one_month.shift(1)) - 1
        one_month_return = round(daily_one_month_return.mean() * 30, 6)


        hqm_dataframe = hqm_dataframe.append(
                                            pd.Series([t, 
                                                       price,
                                                       'N/A',
                                                       one_year_return,
                                                       'N/A',
                                                       six_month_return,
                                                       'N/A',
                                                       three_month_return,
                                                       'N/A',
                                                       one_month_return,
                                                       'N/A',
                                                       'N/A'
                                                       ], 
                                                      index = my_columns), 
                                            ignore_index = True)
    except IndexError:
        continue

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
hqm_dataframe

Ticker   Price Number of Shares to Buy  One-Year Price Return  \
0      MMM  148.48                     N/A              -0.226680   
1      AOS   61.50                     N/A              -0.087659   
2      ABT  109.00                     N/A              -0.078666   
3     ABBV  140.34                     N/A               0.272443   
4     ABMD  293.77                     N/A              -0.063435   
..     ...     ...                     ...                    ...   
496    YUM  118.50                     N/A              -0.076276   
497   ZBRA  330.32                     N/A              -0.464680   
498    ZBH  113.16                     N/A              -0.183395   
499   ZION   54.09                     N/A               0.079302   
500    ZTS  174.01                     N/A              -0.098017   

    One-Year Return Percentile  Six-Month Price Return  \
0                          N/A               -0.068995   
1                          N/A               -0.216875   
2                          N/A               -0.196511   
3                          N/A                0.023351   
4                          N/A                0.115337   
..                         ...                     ...   
496                        N/A               -0.040732   
497                        N/A               -0.526782   
498                        N/A                0.103163   
499                        N/A               -0.337529   
500                        N/A               -0.168962   

    Six-Month Return Percentile  Three-Month Price Return  \
0                           N/A                 -0.012750   
1                           N/A                  0.068158   
2                           N/A                  0.039795   
3                           N/A                 -0.084620   
4                           N/A                  0.369069   
..                          ...                       ...   
496                         N/A                  0.124176   
497                         N/A                  0.047136   
498                         N/A                 -0.006931   
499                         N/A                 -0.000117   
500                         N/A                  0.126003   

    Three-Month Return Percentile  One-Month Price Return  \
0                             N/A                0.215139   
1                             N/A                0.135791   
2                             N/A                0.010973   
3                             N/A               -0.114411   
4                             N/A                0.176902   
..                            ...                     ...   
496                           N/A                0.026823   
497                           N/A                0.201171   
498                           N/A                0.140470   
499                           N/A                0.110648   
500                           N/A               -0.017916   

    One-Month Return Percentile HQM Score  
0                           N/A       N/A  
1                           N/A       N/A  
2                           N/A       N/A  
3                           N/A       N/A  
4                           N/A       N/A  
..                          ...       ...  
496                         N/A       N/A  
497                         N/A       N/A  
498                         N/A       N/A  
499                         N/A       N/A  
500                         N/A       N/A  

[501 rows x 12 columns]

# Calculating Momentum Percentiles

In [18]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], 
                                                                                             hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.161677
1      0.343313
2      0.371257
3      0.894212
4       0.39521
         ...   
496    0.373253
497    0.035928
498    0.195609
499    0.656687
500    0.321357
Name: One-Year Return Percentile, Length: 501, dtype: object
0       0.45509
1      0.207585
2      0.235529
3       0.62475
4      0.768463
         ...   
496    0.522954
497    0.027944
498    0.756487
499      0.0998
500    0.271457
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.323353
1      0.552894
2      0.491018
3      0.177645
4      0.958084
         ...   
496    0.674651
497    0.508982
498    0.343313
499    0.361277
500    0.680639
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.810379
1       0.59481
2      0.173653
3      0.035928
4      0.702595
         ...   
496    0.217565
497    0.782435
498    0.608782
499    0.502994
500    0.105788
Name: One-Month Return Percentile, Length: 501, dtype: object


Ticker   Price Number of Shares to Buy  One-Year Price Return  \
0      MMM  148.48                     N/A              -0.226680   
1      AOS   61.50                     N/A              -0.087659   
2      ABT  109.00                     N/A              -0.078666   
3     ABBV  140.34                     N/A               0.272443   
4     ABMD  293.77                     N/A              -0.063435   
..     ...     ...                     ...                    ...   
496    YUM  118.50                     N/A              -0.076276   
497   ZBRA  330.32                     N/A              -0.464680   
498    ZBH  113.16                     N/A              -0.183395   
499   ZION   54.09                     N/A               0.079302   
500    ZTS  174.01                     N/A              -0.098017   

    One-Year Return Percentile  Six-Month Price Return  \
0                     0.161677               -0.068995   
1                     0.343313               -0.216875   
2                     0.371257               -0.196511   
3                     0.894212                0.023351   
4                      0.39521                0.115337   
..                         ...                     ...   
496                   0.373253               -0.040732   
497                   0.035928               -0.526782   
498                   0.195609                0.103163   
499                   0.656687               -0.337529   
500                   0.321357               -0.168962   

    Six-Month Return Percentile  Three-Month Price Return  \
0                       0.45509                 -0.012750   
1                      0.207585                  0.068158   
2                      0.235529                  0.039795   
3                       0.62475                 -0.084620   
4                      0.768463                  0.369069   
..                          ...                       ...   
496                    0.522954                  0.124176   
497                    0.027944                  0.047136   
498                    0.756487                 -0.006931   
499                      0.0998                 -0.000117   
500                    0.271457                  0.126003   

    Three-Month Return Percentile  One-Month Price Return  \
0                        0.323353                0.215139   
1                        0.552894                0.135791   
2                        0.491018                0.010973   
3                        0.177645               -0.114411   
4                        0.958084                0.176902   
..                            ...                     ...   
496                      0.674651                0.026823   
497                      0.508982                0.201171   
498                      0.343313                0.140470   
499                      0.361277                0.110648   
500                      0.680639               -0.017916   

    One-Month Return Percentile HQM Score  
0                      0.810379       N/A  
1                       0.59481       N/A  
2                      0.173653       N/A  
3                      0.035928       N/A  
4                      0.702595       N/A  
..                          ...       ...  
496                    0.217565       N/A  
497                    0.782435       N/A  
498                    0.608782       N/A  
499                    0.502994       N/A  
500                    0.105788       N/A  

[501 rows x 12 columns]

 # Calculating the HQM Score

In [19]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

# Selecting the 50 Best Momentum Stocks

In [20]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

# Calculating the Number of Shares to Buy

In [21]:
#Assuming portfolio size of $10,000,000
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [22]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\joshu\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Ticker   Price Number of Shares to Buy  One-Year Price Return  \
0     MMM  148.48                    1320              -0.226680   
1     AOS   61.50                    3188              -0.087659   
2     ABT  109.00                    1798              -0.078666   
3    ABBV  140.34                    1397               0.272443   
4    ABMD  293.77                     667              -0.063435   
5     ACN  310.33                     631               0.025911   
6    ATVI   81.00                    2420               0.060452   
7     ADM   82.49                    2376               0.372024   
8    ADBE  434.34                     451              -0.289987   
9     ADP  248.61                     788               0.203752   
10    AAP  194.29                    1009               0.022025   
11    AES   23.87                    8214               0.064138   
12    AFL   59.58                    3291               0.110471   
13      A  132.52                    1479              -0.113506   
14    APD  261.77                     749               0.000842   
15   AKAM   96.38                    2034              -0.142294   
16    ALK   46.80                    4189              -0.111901   
17    ALB  244.84                     800               0.193705   
18    ARE  162.52                    1206              -0.193084   
19   ALGN  293.06                     669              -0.726819   
20   ALLE  102.87                    1906              -0.236227   
21    LNT   61.05                    3211               0.055938   
22    ALL  118.29                    1657              -0.058056   
23  GOOGL  117.30                    1671              -0.100504   
24   GOOG  118.14                    1659              -0.101820   
25     MO   44.22                    4434               0.034139   
26   AMZN  139.41                    1406              -0.086791   
27   AMCR   12.39                   15825               0.114655   
28    AMD  100.07                    1959               0.090615   
29    AEE   91.89                    2133               0.089307   
30    AAL   15.02                   13054              -0.156473   
31    AEP  100.08                    1959               0.163962   
32    AXP  157.28                    1246              -0.009403   
33    AIG   52.27                    3751               0.082444   
34    AMT  273.00                     718               0.030941   
35    AWK  155.15                    1263              -0.088545   
36    AMP  270.17                     725               0.086199   
37    ABC  140.62                    1394               0.195695   
38    AME  125.66                    1560              -0.049469   
39   AMGN  247.29                     792               0.132675   
40    APH   77.58                    2527               0.100034   
41    ADI  175.86                    1114               0.095979   
42   ANSS  282.65                     693              -0.203536   
43    AON  289.83                     676               0.136326   
44    APA   32.21                    6087               0.779114   
45   AAPL  164.87                    1189               0.173371   
46   AMAT  107.83                    1818              -0.162009   
47   APTV  102.48                    1913              -0.374276   
48   ANET  124.71                    1572               0.378939   
49    AJG  179.39                    1093               0.289406   
50    AIZ  158.07                    1240               0.033855   

   One-Year Return Percentile  Six-Month Price Return  \
0                    0.161677               -0.068995   
1                    0.343313               -0.216875   
2                    0.371257               -0.196511   
3                    0.894212                0.023351   
4                     0.39521                0.115337   
5                    0.556886               -0.100988   
6                    0.618762                0.041883   
7         

# Formatting Our Excel Output

In [23]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0.0000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [26]:
writer.save()